In [ ]:
import numpy as np
import pandas as pd
import unittest
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

### class Laguerre

In [ ]:
class Laguerre:
    def __init__(self, beta=2, sigma=4):
        self.beta = beta
        self.sigma = sigma
        
    @property
    def beta(self):
        return self.__beta
        
    @property
    def sigma(self):
        return self.__sigma
        
    @beta.setter
    def beta(self, value):
        if value > 0:
            self.__beta = value
        else:
            raise ValueError("Beta має бути додатнім")
            
    @sigma.setter
    def sigma(self, value):
        if value > 0:
            self.__sigma = value
        else:
            raise ValueError("Sigma має бути додатнім")

    def laguerre_function(self, t, n):
        """Обчислює функцію Лагерра порядку n у t. Повертає float."""
        
        L_0 = np.sqrt(self.sigma) * np.exp(-self.beta / 2 * t)
        if n == 0:
            return L_0

        L_1 = np.sqrt(self.sigma) * (1 - self.sigma * t) * np.exp(-self.beta / 2 * t)
        if n == 1:
            return L_1

        L_nm2 = L_0
        L_nm1 = L_1
        for k in range(2, n + 1):
            L_n = ((2 * k - 1 - self.sigma * t) / k) * L_nm1 - ((k - 1) / k) * L_nm2
            L_nm2, L_nm1 = L_nm1, L_n

        return L_n

    def tabulate_laguerre(self, n, T=10, num_points=100):
        """Табулює функції Лагерра від 0 до n на [0, T]. Повертає pd.DataFrame."""
        
        t_values = np.linspace(0, T, num_points)
        laguerre_values = {
            f"l_{k}(t)": [self.laguerre_function(t, k) for t in t_values]
            for k in range(n + 1)
        }
        laguerre_values['t'] = t_values
        data = pd.DataFrame(laguerre_values)
        return data

    def find_cutoff_T(self, N, epsilon=1e-3, initial_T=1, step=0.1):
        """Знаходить T, де |l_n(T)| < epsilon. Повертає tuple (float, pd.DataFrame)."""
        
        T = initial_T
        while True:
            valid = True
            values_table = []

            for n in range(N + 1):
                l_value = abs(self.laguerre_function(T, n))
                values_table.append({"n": n, "T": T, "|l_n(T)|": l_value})
                if l_value >= epsilon:
                    valid = False

            if valid:
                df = pd.DataFrame(values_table)
                return T, df

            T += step

    def laguerre_integral_simpson(self, f, k, alpha, T, epsilon=1e-6, num_points=1000):
        """Обчислює інтеграл методом Сімпсона. Повертає float."""
        
        if num_points % 2 != 0:
            num_points += 1

        t_values = np.linspace(0, T, num_points + 1)
        h = T / num_points

        integrand = [
            f(t) * self.laguerre_function(t, k) * np.exp(-alpha * t)
            for t in t_values
        ]

        integral = (
            h / 3 * (
                integrand[0]
                + 2 * sum(integrand[2:-1:2])
                + 4 * sum(integrand[1::2])
                + integrand[-1]
            )
        )
        return integral

    def inverse_laguerre_transform(self, hN, t, alpha=None):
        """Обчислює обернене перетворення Лагерра. Повертає float."""
        if alpha is None:
            alpha = self.beta
        return sum(hN[k] * self.laguerre_function(t, k) for k in range(len(hN)))
    
    def laguerre_transform(self, f, N, alpha=None, T=2 * np.pi, epsilon=1e-6):
        """Обчислює пряме перетворення Лагерра. Повертає list."""
        if alpha is None:
            alpha = self.beta  
        return [
            self.laguerre_integral_simpson(f, k, alpha, T, epsilon)
            for k in range(N + 1)
        ]


### class for testing Laguerre

In [ ]:
class TestLaguerreFunctions(unittest.TestCase):
    def setUp(self):
        self.laguerre = Laguerre(beta=2, sigma=4)

    def test_laguerre_function(self):
        result = self.laguerre.laguerre_function(t=1, n=2)
        self.assertAlmostEqual(result, 0.73576, places=5)

    def test_tabulate_laguerre(self):
        df = self.laguerre.tabulate_laguerre(n=2, T=10, num_points=100)
        self.assertEqual(df.shape[0], 100)  
        self.assertEqual(df.shape[1], 4)    
        self.assertIn('t', df.columns)
        self.assertIn('l_2(t)', df.columns)
        
        self.assertTrue(all(df['t'] >= 0))
        self.assertTrue(all(df['t'] <= 10))

    def test_laguerre_integral(self):
        f = lambda t: t**2
        result = self.laguerre.laguerre_integral_simpson(f=f, k=2, alpha=4, T=10)        
        self.assertAlmostEqual(result, 0.00128, places=5)

    def test_laguerre_transform(self):
        f_task5 = lambda t: np.sin(t - np.pi / 2) + 1 if 0 <= t <= 2 * np.pi else 0
        coefficients = self.laguerre.laguerre_transform(f=f_task5, N=20, alpha=4, T=2 * np.pi)
        self.assertEqual(len(coefficients), 21)  
        
        self.assertTrue(all(isinstance(c, (float, np.floating)) for c in coefficients))
        
        self.assertGreater(coefficients[0], 0)

    def test_inverse_laguerre_transform(self):
        coefficients = [0.5] * 10
        reconstructed_value = self.laguerre.inverse_laguerre_transform(hN=coefficients, t=5, alpha=4)
        self.assertIsInstance(reconstructed_value, (float, np.floating))
        
        self.assertNotEqual(reconstructed_value, 0)
        
        self.assertTrue(-10 < reconstructed_value < 10)

    
    def test_parameter_validation(self):
        with self.assertRaises(ValueError):
            Laguerre(beta=-1, sigma=4)
        with self.assertRaises(ValueError):
            Laguerre(beta=2, sigma=-1)
        
        lag = Laguerre()
        with self.assertRaises(ValueError):
            lag.beta = -1
        with self.assertRaises(ValueError):
            lag.sigma = -2

    
    def test_find_cutoff_T(self):
        T, df = self.laguerre.find_cutoff_T(N=2, epsilon=1e-3, initial_T=1, step=0.1)
        self.assertIsInstance(T, float)
        self.assertGreater(T, 0)
        self.assertEqual(len(df), 3) 
        self.assertTrue(all(df['|l_n(T)|'] < 1e-3))

if __name__ == '__main__':
    unittest.main(argv=[''], exit = False)

### class Visualization

In [ ]:
class Visualization:
    def __init__(self, laguerre_instance):
        self.laguerre_instance = laguerre_instance
        self.output_area = widgets.Output()
        self.create_widgets()
        self.display_widgets()
    
    def create_widgets(self):
        self.n_widget = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n")
        self.n_widget.style.handle_color = 'mediumvioletred'

        self.beta_widget = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=2, description="Beta")
        self.beta_widget.style.handle_color = 'mediumvioletred'

        self.sigma_widget = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=4, description="Sigma")
        self.sigma_widget.style.handle_color = 'mediumvioletred'

        self.T_widget = widgets.FloatSlider(min=0.1, max=20, step=0.1, value=10, description="T")
        self.T_widget.style.handle_color = 'mediumvioletred'

        self.num_points_widget = widgets.IntSlider(min=10, max=1000, step=10, value=100, description="Points")
        self.num_points_widget.style.handle_color = 'mediumvioletred'

        self.epsilon_widget = widgets.FloatSlider(min=1e-5, max=1e-2, step=1e-5, value=1e-3, description="Epsilon")
        self.epsilon_widget.style.handle_color = 'mediumvioletred'

        self.function_input = widgets.Text(value="sin(t) + 1", description="Функція f(t):")

        self.btn_compute = widgets.Button(description="Обчислити функцію Лагерра", button_style="primary")
        self.btn_compute.style.button_color = 'mediumvioletred'
        self.btn_compute.on_click(self.compute_laguerre)

        self.btn_plot = widgets.Button(description="Побудувати графік", button_style="primary")
        self.btn_plot.style.button_color = 'mediumvioletred'
        self.btn_plot.on_click(self.plot_laguerre)

        self.btn_find_T = widgets.Button(description="Знайти T", button_style="primary")
        self.btn_find_T.style.button_color = 'mediumvioletred'
        self.btn_find_T.on_click(self.find_cutoff_T)

        self.btn_integral = widgets.Button(description="Обчислити інтеграл", button_style="primary")
        self.btn_integral.style.button_color = 'mediumvioletred'
        self.btn_integral.on_click(self.compute_integral)

        self.btn_transform = widgets.Button(description="Перетворення Лагерра", button_style="primary")
        self.btn_transform.style.button_color = 'mediumvioletred'
        self.btn_transform.on_click(self.compute_transform)

        self.btn_inverse_transform = widgets.Button(description="Обернене перетворення Лагерра", button_style="primary")
        self.btn_inverse_transform.style.button_color = 'mediumvioletred'
        self.btn_inverse_transform.on_click(self.compute_inverse_transform)

        self.btn_clear_output = widgets.Button(description="Очистити вивід", button_style="danger")
        self.btn_clear_output.style.button_color = 'grey'
        self.btn_clear_output.on_click(self.clear_output)

        self.btn_compare_plots = widgets.Button(description="Порівняти графіки", button_style="primary")
        self.btn_compare_plots.style.button_color = 'mediumvioletred'
        self.btn_compare_plots.on_click(self.compare_plots)
    
    def display_widgets(self):
        display(self.n_widget, self.beta_widget, self.sigma_widget, self.T_widget, self.num_points_widget, self.epsilon_widget, self.function_input)
        display(self.btn_compute, self.btn_plot, self.btn_find_T, self.btn_integral, self.btn_transform, self.btn_inverse_transform, self.btn_compare_plots, self.btn_clear_output)
        display(self.output_area)
    
    def get_function_context(self):
        """Повертає контекст для eval із математичними функціями та константами."""
        return {
            "t": None,
            "np": np,
            "sin": np.sin,
            "cos": np.cos,
            "tan": np.tan,
            "exp": np.exp,
            "log": np.log,
            "sqrt": np.sqrt,
            "pi": np.pi,
            "e": np.e
        }
    
    def plot_laguerre_functions(self, n, T=10, num_points=100):
        table_df = self.laguerre_instance.tabulate_laguerre(n, T, num_points)
        with self.output_area:
            plt.figure(figsize=(8, 5))
            for k in range(n + 1):
                plt.plot(table_df['t'], table_df[f"l_{k}(t)"], label=f"$l_{{{k}}}(t)$")
            plt.xlabel('t values')
            plt.ylabel(f"$l_{{n}}(t)$ values")
            plt.title(f"Laguerre Functions Plot for n = {n}")
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.legend()
            plt.show()
    
    def compute_laguerre(self, b):
        with self.output_area:
            print("Laguerre Function Sample:")
            print(self.laguerre_instance.laguerre_function(4, self.n_widget.value))
    
    def plot_laguerre(self, b):
        with self.output_area:
            print("\nПобудова графіка функцій Лагерра:")
        self.plot_laguerre_functions(self.n_widget.value, self.T_widget.value, self.num_points_widget.value)
    
    def find_cutoff_T(self, b):
        with self.output_area:
            cutoff_T, df = self.laguerre_instance.find_cutoff_T(self.n_widget.value, self.epsilon_widget.value)
            print(f"\nCutoff T: {cutoff_T}")
            display(df)
    
    def compute_integral(self, b):
        with self.output_area:
            try:
                context = self.get_function_context()
                f = lambda t: eval(self.function_input.value, dict(context, t=t))
                integral_value = self.laguerre_instance.laguerre_integral_simpson(f, 2, self.sigma_widget.value, self.T_widget.value)
                print(f"\nLaguerre Integral Value: {integral_value}")
            except Exception as e:
                print(f"Помилка при обчисленні функції: {e}")
                print("Будь ласка, введіть коректний математичний вираз.")
    
    def compute_transform(self, b):
        with self.output_area:
            try:
                context = self.get_function_context()
                f = lambda t: eval(self.function_input.value, dict(context, t=t))
                coefficients = self.laguerre_instance.laguerre_transform(f, self.n_widget.value, self.beta_widget.value, self.T_widget.value, self.epsilon_widget.value)
                print(f"\nLaguerre Transform Coefficients: {coefficients}")
            except Exception as e:
                print(f"Помилка при обчисленні функції: {e}")
                print("Будь ласка, введіть коректний математичний вираз.")
    
    def compute_inverse_transform(self, b):
        with self.output_area:
            try:
                context = self.get_function_context()
                f = lambda t: eval(self.function_input.value, dict(context, t=t))
                hN = self.laguerre_instance.laguerre_transform(f, self.n_widget.value, self.beta_widget.value, self.T_widget.value, self.epsilon_widget.value)
                t_value = 4
                reconstructed_value = self.laguerre_instance.inverse_laguerre_transform(hN, t_value, self.beta_widget.value)
                print(f"\nОбернене перетворення Лагерра для t={t_value}: {reconstructed_value}")
            except Exception as e:
                print(f"Помилка при обчисленні функції: {e}")
                print("Будь ласка, введіть коректний математичний вираз.")
    
    def clear_output(self, b):
        self.output_area.clear_output()
    
    def compare_plots(self, b):
        with self.output_area:
            self.laguerre_instance.beta = self.beta_widget.value
            self.laguerre_instance.sigma = self.sigma_widget.value

            n = self.n_widget.value
            T = self.T_widget.value
            num_points = self.num_points_widget.value
            epsilon = self.epsilon_widget.value

            try:
                context = self.get_function_context()
                f = lambda t: eval(self.function_input.value, dict(context, t=t))

                hN = self.laguerre_instance.laguerre_transform(f, n, T=T, epsilon=epsilon)
                
                t_values = np.linspace(0, T, num_points)

                f_values = [f(t) for t in t_values]

                inverse_values = [self.laguerre_instance.inverse_laguerre_transform(hN, t) for t in t_values]

                plt.figure(figsize=(10, 6))
                plt.plot(t_values, f_values, label="f(t) (оригінальна функція)", color='blue')
                plt.plot(t_values, inverse_values, label="Обернене перетворення", color='red', linestyle='-.')
                plt.xlabel('t')
                plt.ylabel('Значення')
                plt.title(f"Порівняння функцій для n = {n}")
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.legend()
                plt.show()
            except Exception as e:
                print(f"Помилка при обчисленні функції: {e}")
                print("Будь ласка, введіть коректний математичний вираз (наприклад, 't + 2', 'sin(t)', 'exp(t)').")

### Execution


In [ ]:

visualization = Visualization(Laguerre())


### CSV Visualization

In [ ]:
class CSV_Visualization:
    def __init__(self, laguerre_instance):
        self.laguerre_instance = laguerre_instance
        self.output_area = widgets.Output()
        self.loaded_data = None  
        self.loaded_tabulation = None  
        self.loaded_cutoff_table = None
        self.create_widgets()
        self.display_widgets()
    
    def create_widgets(self):
        self.n_widget = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n")
        self.n_widget.style.handle_color = 'mediumvioletred'

        self.beta_widget = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=2.0, description="Beta")
        self.beta_widget.style.handle_color = 'mediumvioletred'

        self.sigma_widget = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=4, description="Sigma")
        self.sigma_widget.style.handle_color = 'mediumvioletred'

        self.T_widget = widgets.FloatSlider(min=0.1, max=20, step=0.1, value=10, description="T")
        self.T_widget.style.handle_color = 'mediumvioletred'

        self.num_points_widget = widgets.IntSlider(min=10, max=1000, step=10, value=100, description="Points")
        self.num_points_widget.style.handle_color = 'mediumvioletred'

        self.epsilon_widget = widgets.FloatSlider(min=1e-5, max=1e-2, step=1e-5, value=1e-3, description="Epsilon")
        self.epsilon_widget.style.handle_color = 'mediumvioletred'

        self.function_input = widgets.Text(value="sin(t) + 1", description="Функція f(t):")

        self.btn_compute = widgets.Button(description="Обчислити функцію Лагерра", button_style="primary")
        self.btn_compute.style.button_color = 'mediumvioletred'
        self.btn_compute.on_click(self.compute_laguerre)

        self.btn_plot = widgets.Button(description="Побудувати графік", button_style="primary")
        self.btn_plot.style.button_color = 'mediumvioletred'
        self.btn_plot.on_click(self.plot_laguerre)

        self.btn_find_T = widgets.Button(description="Знайти T", button_style="primary")
        self.btn_find_T.style.button_color = 'mediumvioletred'
        self.btn_find_T.on_click(self.find_cutoff_T)

        self.btn_integral = widgets.Button(description="Обчислити інтеграл", button_style="primary")
        self.btn_integral.style.button_color = 'mediumvioletred'
        self.btn_integral.on_click(self.compute_integral)

        self.btn_transform = widgets.Button(description="Перетворення Лагерра", button_style="primary")
        self.btn_transform.style.button_color = 'mediumvioletred'
        self.btn_transform.on_click(self.compute_transform)

        self.btn_inverse_transform = widgets.Button(description="Обернене перетворення Лагерра", button_style="primary")
        self.btn_inverse_transform.style.button_color = 'mediumvioletred'
        self.btn_inverse_transform.on_click(self.compute_inverse_transform)

        self.btn_clear_output = widgets.Button(description="Очистити вивід", button_style="danger")
        self.btn_clear_output.style.button_color = 'grey'
        self.btn_clear_output.on_click(self.clear_output)

        self.btn_compare_plots = widgets.Button(description="Порівняти графіки", button_style="primary")
        self.btn_compare_plots.style.button_color = 'mediumvioletred'
        self.btn_compare_plots.on_click(self.compare_plots)

        self.btn_save_csv = widgets.Button(description="Зберегти в CSV", button_style="success")
        self.btn_save_csv.style.button_color = 'mediumvioletred'
        self.btn_save_csv.on_click(self.save_to_csv)

        self.btn_load_csv = widgets.Button(description="Завантажити з CSV", button_style="warning")
        self.btn_load_csv.style.button_color = 'mediumvioletred'
        self.btn_load_csv.on_click(self.load_from_csv)

    def display_widgets(self):
        display(self.n_widget, self.beta_widget, self.sigma_widget, self.T_widget, self.num_points_widget, self.epsilon_widget, self.function_input)
        display(self.btn_compute, self.btn_plot, self.btn_find_T, self.btn_integral, self.btn_transform, self.btn_inverse_transform, self.btn_compare_plots, self.btn_save_csv, self.btn_load_csv, self.btn_clear_output)
        display(self.output_area)
    
    def get_function_context(self):
        """Повертає контекст для eval із математичними функціями та константами."""
        return {
            "t": None,
            "np": np,
            "sin": np.sin,
            "cos": np.cos,
            "tan": np.tan,
            "exp": np.exp,
            "log": np.log,
            "sqrt": np.sqrt,
            "pi": np.pi,
            "e": np.e
        }
    
    def plot_laguerre_functions(self, n, T=10, num_points=100):
        with self.output_area:
            plt.figure(figsize=(8, 5))
            if self.loaded_tabulation is not None and len(self.loaded_tabulation) > 0:
                t_values = self.loaded_tabulation['t']
                for k in range(n + 1):
                    col = f"l_{k}(t)"
                    if col in self.loaded_tabulation:
                        plt.plot(t_values, self.loaded_tabulation[col], label=f"$l_{{{k}}}(t)$")
            else:
                table_df = self.laguerre_instance.tabulate_laguerre(n, T, num_points)
                for k in range(n + 1):
                    plt.plot(table_df['t'], table_df[f"l_{k}(t)"], label=f"$l_{{{k}}}(t)$")
            plt.xlabel('t values')
            plt.ylabel(f"$l_{{n}}(t)$ values")
            plt.title(f"Laguerre Functions Plot for n = {n}")
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.legend()
            plt.show()
    
    def compute_laguerre(self, b):
        with self.output_area:
            if self.loaded_data is not None and 'LaguerreFunctionResult' in self.loaded_data:
                print("Laguerre Function Sample (from loaded data):")
                print(self.loaded_data['LaguerreFunctionResult'])
    
    def plot_laguerre(self, b):
        with self.output_area:
            print("\nПобудова графіка функцій Лагерра:")
        self.plot_laguerre_functions(self.n_widget.value, self.T_widget.value, self.num_points_widget.value)
    
    def find_cutoff_T(self, b):
        with self.output_area:
            if self.loaded_data is not None and 'CutoffT' in self.loaded_data and self.loaded_cutoff_table is not None:
                print(f"\nCutoff T (from loaded data): {self.loaded_data['CutoffT']}")
                display(self.loaded_cutoff_table)

    def compute_integral(self, b):
        with self.output_area:
            if self.loaded_data is not None and 'Integral' in self.loaded_data:
                print(f"\nLaguerre Integral Value (from loaded data): {self.loaded_data['Integral']}")

    def compute_transform(self, b):
        with self.output_area:
            if self.loaded_data is not None and 'Transform' in self.loaded_data:
                print(f"\nLaguerre Transform Coefficients (from loaded data): {self.loaded_data['Transform']}")

    def compute_inverse_transform(self, b):
        with self.output_area:
            if self.loaded_data is not None and 'InverseTransform' in self.loaded_data:
                print(f"\nОбернене перетворення Лагерра для t=4 (from loaded data): {self.loaded_data['InverseTransform']}")
    
    def clear_output(self, b):
        self.output_area.clear_output()
    
    def compare_plots(self, b):
        with self.output_area:
            if self.loaded_data is None or 'Transform' not in self.loaded_data:
                print("Помилка: Дані з CSV не завантажені. Спочатку натисніть 'Завантажити з CSV'.")
                return

            try:
                n = self.n_widget.value
                T = self.T_widget.value
                num_points = self.num_points_widget.value
                function_str = self.function_input.value

                self.laguerre_instance.beta = self.beta_widget.value
                self.laguerre_instance.sigma = self.sigma_widget.value

                context = self.get_function_context()
                f = lambda t: eval(function_str, dict(context, t=t))

                hN_csv = [float(x) for x in self.loaded_data['Transform'].replace(',', '.').split(';')]

                t_values = np.linspace(0, T, num_points)

                f_values = [f(t) for t in t_values]

                inverse_values_csv = [self.laguerre_instance.inverse_laguerre_transform(hN_csv, t) for t in t_values]

                plt.figure(figsize=(10, 6))
                plt.plot(t_values, f_values, label="f(t) (оригінальна функція)", color='blue')
                plt.plot(t_values, inverse_values_csv, label="Обернене (CSV)", color='red', linestyle='-.')
                plt.xlabel('t')
                plt.ylabel('Значення')
                plt.title(f"Порівняння функцій для n = {n} (CSV vs Python)")
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.legend()
                plt.show()

            except Exception as e:
                print(f"Помилка при побудові графіків: {e}")
                print("Переконайтеся, що дані з CSV коректні та введіть коректний математичний вираз.")

    def get_function_context_to_csv(self):
        """Повертає контекст для NCalc із математичними функціями та константами."""
        return {
            "variable": "t",  # Змінна, яку використовує NCalc
            "functions": ["Sin", "Cos", "Tan", "Exp", "Log", "Sqrt"],  # Доступні функції в NCalc
            "constants": {
                "pi": "PI",
                "e": "E"
            }
        }

    def get_function_context_from_csv(self, expr):
        replacements = {
            "Sin": "sin",
            "Cos": "cos",
            "Tan": "tan",
            "Exp": "exp",
            "Log": "log",
            "Sqrt": "sqrt",
            "PI": "pi",
            "E": "e"
        }
        for old, new in replacements.items():
            expr = expr.replace(old, new)
        return expr
    
    def save_to_csv(self, b):
        """Зберігає параметри у CSV-файл із виразом, сумісним із NCalc."""
        with self.output_area:
            try:
                # Отримуємо контекст для перевірки виразу
                context = self.get_function_context_to_csv()
                
                # Отримуємо введений вираз
                function_input = self.function_input.value.strip()
                
                # Перевірка, чи вираз містить лише дозволені функції та змінну
                allowed_functions = context["functions"]
                allowed_variable = context["variable"]
                allowed_constants = list(context["constants"].keys())
                
                # Простий синтаксичний аналіз для перевірки сумісності
                valid = True
                for func in allowed_functions:
                    function_input = function_input.replace(func.lower(), func)  # Перетворюємо на правильний регістр
                for const in allowed_constants:
                    function_input = function_input.replace(const.lower(), const)
                
                # Збираємо параметри
                widget_params = {
                    'n': self.n_widget.value,
                    'beta': round(self.beta_widget.value, 4),
                    'sigma': round(self.sigma_widget.value, 4),
                    'T': round(self.T_widget.value, 4),
                    'num_points': self.num_points_widget.value,
                    'epsilon': round(self.epsilon_widget.value, 6),
                    'function': function_input  
                }

                params_df = pd.DataFrame([widget_params])

                filename = "laguerre_params.csv"
                params_df.to_csv(filename, index=False)
                print(f"Параметри збережено у файл: {filename}")
            except Exception as e:
                print(f"Помилка при збереженні в CSV: {e}")

    def load_from_csv(self, b):
        with self.output_area:
            try:
                filename = "laguerre_results.csv"
                df = pd.read_csv(filename, header=None)

                params = df.iloc[0].to_dict()
                params = {
                    'n': int(params[0]),
                    'beta': float(params[1]),
                    'sigma': float(params[2]),
                    'T': float(params[3]),
                    'num_points': int(params[4]),
                    'epsilon': float(params[5]),
                    'function': params[6]
                }

                self.n_widget.value = params['n']
                self.beta_widget.value = params['beta']
                self.sigma_widget.value = params['sigma']
                self.T_widget.value = params['T']
                self.num_points_widget.value = params['num_points']
                self.epsilon_widget.value = params['epsilon']
                self.function_input.value = self.get_function_context_from_csv(params['function'])

                results_idx = df.index[df[0] == '---'].tolist()[0] + 1
                results = df.iloc[results_idx].to_dict()
                results_dict = {
                    'LaguerreFunctionResult': float(results[0]),
                    'CutoffT': float(results[1]),
                    'Integral': float(results[2]),
                    'Transform': results[3].replace(',', '.'),  
                    'InverseTransform': float(results[4])
                }

                cutoff_start = df.index[df[0] == 'Cutoff Table'].tolist()[0] + 1
                cutoff_headers = df.iloc[cutoff_start].tolist()[:3]

                cutoff_data = df.iloc[cutoff_start + 1:].values
                filtered_cutoff_data = []
                for _, row in enumerate(cutoff_data):
                    row = row[:3]
                    if len(row) != 3:
                        continue
                    if any(x is None for x in row):
                        continue
                    converted_row = []
                    for val in row:
                        val_str = str(val).strip()
                        converted_row.append(float(val_str))
                    filtered_cutoff_data.append(converted_row)
                    
                if filtered_cutoff_data:
                    cutoff_df = pd.DataFrame(filtered_cutoff_data, columns=cutoff_headers)
                else:
                    print("Попередження: Таблиця відсічення порожня або має неправильний формат.")
                    print(f"Відфільтровані дані: {filtered_cutoff_data}")
                    cutoff_df = pd.DataFrame(columns=cutoff_headers)

                
                filename_tab = "tabulation_results.csv"
                with open(filename_tab, 'r') as file:
                    lines_tab = file.readlines()

                n = params['n']
                tabulation_headers = lines_tab[0].strip().split('\t')
                expected_cols = n + 2
                if len(tabulation_headers) != expected_cols:
                    raise ValueError(f"Неправильна кількість заголовків у табуляції {filename_tab}: очікується {expected_cols}, отримано {len(tabulation_headers)}")

                tabulation_data = []
                for line in lines_tab[1:]:
                    row = line.strip().split('\t')
                    if row and len(row) == len(tabulation_headers):
                        try:
                            tabulation_data.append([float(x) for x in row])
                        except ValueError:
                            continue
                tabulation_df = pd.DataFrame(tabulation_data, columns=tabulation_headers)
                if 't' in tabulation_df.columns:
                    tabulation_df = tabulation_df[['t'] + [col for col in tabulation_df.columns if col != 't']]

                # Зберігаємо всі дані
                self.loaded_data = results_dict
                self.loaded_tabulation = tabulation_df
                self.loaded_cutoff_table = cutoff_df

                print(f"Дані успішно завантажено з файлів: {filename} та {filename_tab}")
                print("Параметри:")
                display(pd.DataFrame([params]))
                print("Результати:")
                display(pd.DataFrame([results_dict]))
                print("Табуляція:")
                display(self.loaded_tabulation)
                print("Таблиця відсічення:")
                display(self.loaded_cutoff_table)


            except Exception as e:
                print(f"Помилка при завантаженні з CSV: {e}")
                print("Переконайтеся, що файл 'laguerre_results.csv' існує та має правильний формат.")



In [ ]:
viz = CSV_Visualization(Laguerre())